In [5]:
import tensorflow as tf

# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the images to the range 
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Add a channel dimension (for convolutional layers)
train_images = train_images.reshape(train_images.shape + (1,))
test_images = test_images.reshape(test_images.shape + (1,))

(10000, 28, 28)


In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

# Evaluate the baseline model
_, baseline_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f'Baseline model accuracy: {baseline_accuracy:.4f}')

# Save the baseline model (optional, but good practice)
model.save('mnist_baseline.h5')

Epoch 1/5


/Users/berkdelibalta/miniconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8848 - loss: 0.4059
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9759 - loss: 0.0838
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9804 - loss: 0.0649
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9851 - loss: 0.0495
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9878 - loss: 0.0429


Baseline model accuracy: 0.9834


In [33]:
# Load the saved baseline model
loaded_model = tf.keras.models.load_model('mnist_baseline.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)

# Enable default optimizations for weight quantization (dynamic range quantization)
converter.optimizations = tflite_model_dynamic_range = converter.convert()

# Save the dynamically quantized model
with open('mnist_quantized_dynamic_range.tflite', 'wb') as f:
    f.write(tflite_model_dynamic_range)

print("Dynamically quantized model saved.")

# For full integer quantization (weights and activations):
def representative_dataset():
    for image in train_images[:100]:  # Use a small subset of the training data
        yield [image.reshape(1, 28, 28, 1)]

converter_full_int = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter_full_int.optimizations = [tf.lite.Optimize.DEFAULT]
converter_full_int.representative_dataset = representative_dataset
converter_full_int.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_full_int.inference_input_type = tf.uint8  # or tf.int8
converter_full_int.inference_output_type = tf.uint8 # or tf.int8
tflite_model_full_int = converter_full_int.convert()

# Save the fully integer quantized model
with open('mnist_quantized_full_int.tflite', 'wb') as f:
    f.write(tflite_model_full_int)

print("Fully integer quantized model saved.")

INFO:tensorflow:Assets written to: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv/assets


INFO:tensorflow:Assets written to: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv/assets


Saved artifact at '/var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13691988176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13653262608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13691990864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13691990672: TensorSpec(shape=(), dtype=tf.resource, name=None)
Dynamically quantized model saved.
INFO:tensorflow:Assets written to: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpqw7bvs2c/assets


W0000 00:00:1745252025.836895 14062007 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1745252025.836904 14062007 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-04-21 12:13:45.837003: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv
2025-04-21 12:13:45.837227: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-21 12:13:45.837231: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv
2025-04-21 12:13:45.838660: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-21 12:13:45.847729: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpvrpiv0dv
2025-04-21 12:13:45.851114: I tensorflow/cc/saved_model/loader.c

Saved artifact at '/var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpqw7bvs2c'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13691988176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13653262608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13691990864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13691990672: TensorSpec(shape=(), dtype=tf.resource, name=None)


/Users/berkdelibalta/miniconda3/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1745252025.941610 14062007 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1745252025.941619 14062007 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2025-04-21 12:13:45.941713: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpqw7bvs2c
2025-04-21 12:13:45.941910: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-21 12:13:45.941915: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/rb/8y6my9n97ms5wwj0qv5wfk4h0000gn/T/tmpqw7bvs2c
2025-04-21 12:13:45.943260: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-21 12:13:45.952094: I tensorf

Fully integer quantized model saved.


In [34]:
import numpy as np
import time 

def evaluate_tflite_model(tflite_model, test_images, test_labels):
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    predictions = []  # <-- Initialize predictions as an empty list!

    for image in test_images:
        if input_details[0]['dtype'] == np.uint8:
            input_scale, input_zero_point = input_details[0]['quantization']
            image = image / input_scale + input_zero_point

        image = np.expand_dims(image, axis=0).astype(input_details[0]['dtype'])
        interpreter.set_tensor(input_details[0]['index'], image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(np.argmax(output))
        

    accuracy = np.mean(np.array(predictions) == test_labels)
    return accuracy

# Evaluate the dynamically quantized model
dynamic_range_accuracy = evaluate_tflite_model(tflite_model_dynamic_range, test_images, test_labels)
print(f'Dynamically quantized model accuracy: {dynamic_range_accuracy:.4f}')

# Evaluate the fully integer quantized model
full_int_accuracy = evaluate_tflite_model(tflite_model_full_int, test_images, test_labels)
print(f'Fully integer quantized model accuracy: {full_int_accuracy:.4f}')

import os

# Get the size of the dynamically quantized model
dynamic_range_size = os.path.getsize('mnist_quantized_dynamic_range.tflite') / (1024)
print(f'Dynamically quantized model size: {dynamic_range_size:.2f} KB')

# Get the size of the fully integer quantized model
full_int_size = os.path.getsize('mnist_quantized_full_int.tflite') / (1024)
print(f'Fully integer quantized model size: {full_int_size:.2f} KB')

# Get the size of the baseline model (approximated from the saved.h5 file)
baseline_size = os.path.getsize('mnist_baseline.h5') / (1024)
print(f'Baseline model size (approximate): {baseline_size:.2f} KB')

Dynamically quantized model accuracy: 0.9834
Fully integer quantized model accuracy: 0.9831
Dynamically quantized model size: 214.85 KB
Fully integer quantized model size: 57.20 KB
Baseline model size (approximate): 661.91 KB


In [38]:
# Load baseline Keras model
baseline_model = tf.keras.models.load_model("mnist_baseline.h5")

# Prepare a dummy input
input_data = np.random.rand(1, 28, 28, 1).astype(np.float32)

# Measure time
times = []
for _ in range(1000):
    start_time = time.time()
    _ = baseline_model.predict(input_data, verbose=0)
    end_time = time.time()
    times.append((end_time - start_time) * 1000)  # ms

# Get average time
avg_time_baseline = np.mean(times)
print(f"Baseline Keras Model Average Inference Time: {avg_time_baseline:.4f} ms/sample")


Baseline Keras Model Average Inference Time: 14.4569 ms/sample


In [36]:
def measure_tflite_inference_time(interpreter, input_shape, runs=100, input_dtype=np.float32):
    input_details = interpreter.get_input_details()
    input_data = np.random.random_sample(input_shape)

    if input_dtype == np.int8:
        input_data = (input_data * 127).astype(np.int8)  # Scale to int8 range [-128, 127]
    else:
        input_data = input_data.astype(input_dtype)
    
    # Warm-up
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    times = []
    for _ in range(runs):
        start_time = time.time()
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        end_time = time.time()
        times.append((end_time - start_time) * 1000)  # ms

    avg_time = np.mean(times)
    return avg_time

In [37]:
dynamic_interpreter = tf.lite.Interpreter(model_path="mnist_quantized_dynamic_range.tflite")
dynamic_interpreter.allocate_tensors()

avg_time_tflite = measure_tflite_inference_time(dynamic_interpreter, (1, 28, 28, 1))
print(f"TFLite Dynamic Model Average Inference Time: {avg_time_tflite:.4f} ms/sample")

fullInt_interpreter = tf.lite.Interpreter(model_path="mnist_quantized_dynamic_range.tflite")
fullInt_interpreter.allocate_tensors()

avg_time_tflite = measure_tflite_inference_time(fullInt_interpreter, (1, 28, 28, 1))
print(f"TFLite Full Integer Model Average Inference Time: {avg_time_tflite:.4f} ms/sample")

TFLite Dynamic Model Average Inference Time: 0.0233 ms/sample
TFLite Full Integer Model Average Inference Time: 0.0198 ms/sample
